In [2]:
# 4.4.1 (C=30 kg), vuelos directos (>8 km) y OR-Tools
import os
import pandas as pd
import numpy as np
from ortools.constraint_solver import pywrapcp, routing_enums_pb2

# 1) Parámetros
C     = 30           # capacidad dron [kg]
R_max = 8000         # radio “cercano” [m]
nd    = 3
turns = 8
nv    = nd * turns   # 24 vuelos virtuales

# 2) Matriz de distancias completa
D_full = np.loadtxt('../data/distance_matrix.csv', delimiter=',')

# 3) Demanda split (4.4.2)
dem_split = pd.read_csv('../data/simulated_demands_split.csv')
day0     = dem_split['date'].iloc[0]
day_df   = dem_split[dem_split['date'] == day0].reset_index(drop=True)

# 4) Nombres y coords UTM
paradas_df = pd.read_csv('../data/PARADAS_CON_HABITANTES.csv')
coords_dep = np.array([[570886.225, 4682838.781]])
stop_coords = {r['Name']:(r['stop_x'], r['stop_y']) for _,r in paradas_df.iterrows()}

names  = ['DEPOT_RIBADAVIA']
coords = [coords_dep[0]]
for s in day_df['stop']:
    names.append(s)
    coords.append(stop_coords[s])
coords = np.vstack(coords)

# 5) Matriz local
N        = len(names)
dist_mat = np.linalg.norm(coords[:,None,:] - coords[None,:,:], axis=2)

# 6) Demand list
demand_list = [0] + day_df['simulated_demand_kg'].astype(int).tolist()

# 7) Vuelos directos (>R_max)
far_idx = [i for i in range(1,N) if dist_mat[0,i] > R_max]
directs = []
for j in far_idx:
    d = dist_mat[0,j]
    directs.append({
        'route':      f"DEPOT → {names[j]} → DEPOT",
        'distance_m': int(2*d),
        'distance_km': 2*d/1000,
        'demand_kg':  demand_list[j]
    })
# Numeramos del 1 a num_directs
for i,d in enumerate(directs, start=1):
    d['vehicle'] = i

num_directs = len(directs)

# 8) Subproblema de nudos cercanos
near_idx = [i for i in range(N) if i not in far_idx]
map_idx  = {old:i for i,old in enumerate(near_idx)}
inv_map  = {i:old for old,i in map_idx.items()}

N_sub    = len(near_idx)
sub_dist = np.zeros((N_sub,N_sub), dtype=int)
sub_dem  = []
sub_names= []

for i_new,i_old in inv_map.items():
    sub_names.append(names[i_old])
    sub_dem.append(demand_list[i_old])
    for j_new,j_old in inv_map.items():
        sub_dist[i_new,j_new] = int(dist_mat[i_old,j_old])

near_nv = nv - num_directs

# 9) OR-Tools para cercanos
mgr = pywrapcp.RoutingIndexManager(N_sub, near_nv, map_idx[0])
rt  = pywrapcp.RoutingModel(mgr)

# 10) Callback distancia
def cb_dist(frm,to):
    return sub_dist[mgr.IndexToNode(frm), mgr.IndexToNode(to)]
idx_dist = rt.RegisterTransitCallback(cb_dist)
rt.SetArcCostEvaluatorOfAllVehicles(idx_dist)

# 11) Callback demanda
def cb_dem(frm):
    return sub_dem[mgr.IndexToNode(frm)]
idx_dem = rt.RegisterUnaryTransitCallback(cb_dem)
rt.AddDimensionWithVehicleCapacity(
    idx_dem, 0, [C]*near_nv, True, 'Capacity'
)

# 12) Búsqueda
params = pywrapcp.DefaultRoutingSearchParameters()
params.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
params.time_limit.seconds = 20

# 13) Resolver
sol = rt.SolveWithParameters(params)
if not sol:
    raise RuntimeError("No solución factible para cercanos.")

# 14) Extraer rutas: saltamos el primer idx (el depósito) para no repetirlo
routes = directs.copy()
for v in range(near_nv):
    start_idx = rt.Start(v)
    # avanzamos un paso para omitir la 'visita' inicial al depósito
    idx = sol.Value(rt.NextVar(start_idx))
    prev = map_idx[0]
    dist_v, weight_v = 0, 0
    seq = ['DEPOT_RIBADAVIA']
    while not rt.IsEnd(idx):
        node = mgr.IndexToNode(idx)
        orig = inv_map[node]
        # solo añadimos parada si no es el depot
        if orig != 0:
            seq.append(sub_names[node])
            weight_v += demand_list[orig]
        dist_v += sub_dist[prev, node]
        prev = node
        idx = sol.Value(rt.NextVar(idx))
    # retorno final al depot
    dist_v += sub_dist[prev, map_idx[0]]
    seq.append('DEPOT_RIBADAVIA')

    routes.append({
        'vehicle':    num_directs + v + 1,
        'route':      " → ".join(seq),
        'distance_m': int(dist_v),
        'distance_km': dist_v/1000,
        'demand_kg':  weight_v
    })

# 15) Formateo final y guardado
os.makedirs('../results', exist_ok=True)
df = pd.DataFrame(routes)

# calcular tiempo total estimado (min)
df['total_time_min'] = df['distance_m'] / 15 / 60 + 2

# redondear y ordenar ceros al final
df['distance_km']     = df['distance_km'].round(2)
df['total_time_min']  = df['total_time_min'].round(2)
df['is_zero'] = df['distance_km'] == 0
df = df.sort_values(by=['is_zero','vehicle']).drop(columns=['is_zero']).reset_index(drop=True)

# columnas finales
df_clean = df[['vehicle','route','distance_km','distance_m','demand_kg','total_time_min']]
df_clean.to_csv('../results/routes_mixed.csv', index=False)

print("✅ Rutas 1–24, sin depot duplicado, redondeadas y ceros al final en ../results/routes_mixed.csv")


✅ Rutas 1–24, sin depot duplicado, redondeadas y ceros al final en ../results/routes_mixed.csv
